<a href="https://colab.research.google.com/github/unmo/nlp_for_bert/blob/main/memo/session3/section3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERTの全体像

## BERTの概要

* BERTとは
  * 2018年の後半にGoogleから発表
  * Transfomerがベースとなっている
  * 様々な自然言語処理タスクでファインチューニングが可能
  * 従来の自然言語処理タスクと比較して、高い汎用性

* BERTの入力
  * Tokenベクトル
  * セグメントベクトル(単語の属する文章を表す)
  * ポジションベクトル(単語の位置を表す)


# Transfomerのモデルについて

* Encoder,Decoderモデル
  * BERTではTransfomerのEncoderのみをもちいている
* Encoderの構造
  1. Embedding層により入力文章をベクトルに圧縮
  2. Positional Encoder層によって位置情報を加える
  3. Multi-Head Attention層
  4. normalizationなど
  5. Positionwise fully connected feed-forward network
  6. normalizationなど
  3-6を6回繰り返す
  
* Decoderの構造
  1. Embedding層により入力文章をベクトルに圧縮
  2. Positional Encoder層によって位置情報を加える
  3. Masked-Multi-Head Attention層
  4. normalizationなど
  5. Multi-Head Attention層(Encoderの出力を使用)
  6. normalizationなど
  7. Positionwise fully connected feed-forward network
  8. normalizationなど
  3-8を6回繰り返す
  

# Transfomerの構成要素

## Atentionの種類
* Self-Attention
* SourceTarget-Attention
* Masked Multi-Head Attention
* Multi-Head Attention

## Attentionとは？

* Attentionとは
  * 文章中のどの単語に注目すればいいかを表すスコア
  * Query, Key, Valueの3つのベクトルで計算される
* Query
  * Inputのうち「検索をかけたいもの」
* Key(Memory)
  * 検索対象とQueryの近さを測る
* Value(Memory)
  * Keyに基づき、適切なValueを出力する

## Self-Attention
* InputとMemoryが同一文章であるAttention
* 文法の構造や、単語同士の関係性などを獲得するのに使用される

## SourceTarget-Attention
* InputとMemoryが異なる文章であるAttention
* TransformerではDecoderで使用される

## Multi-Head Attention
* Attentionを平行に並べる
* それぞれのAttentionはHeadと呼ばれる
* それぞれのAttentionをconcatする

## Masked Multi-Head Attention
* 特定のKeyに対してAttention weightを0にする
* TransformerではDecoderで使用されている
* 入力した単語が先読みを防ぐために情報をマスクで遮断する

## Position-wise Feedforward Network
* 2層の全結合ニューラルネットワーク
* 単語の位置ごとに個別の順伝播ネットワーク
* 他の単語との影響関係を排除する
* parameterは全てのネットワークで共通
  * $FFN(x)=max(0, xWz+b1)W2+b2$

## Positional Encoding
* 「単語の位置」の情報を加える
* $PE(pos,2i) = sin(pos/10000^{2i/dmodel})$
* $PE(pos,2i+1) = cos(pos/10000^{2i/dmodel})$
※ pos: 単語の位置, 2i, 2i+1: Embeddingの何番目の次元か, dmodel: 次元数

## Attentionを可視化することで、どのトークンがどのトークンに注意を向けているかを可視化が可能(画像でいうと、画像のどの位置に着目しているか)

# BERTの実装

In [ ]:
!pip install folium==0.2.1
!pip install urllib3==1.25.11
!pip install pytorch-transformers==1.2.0

     |████████████████████████████████| 127 kB 5.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
     |████████████████████████████████| 176 kB 5.3 MB/s 
     |████████████████████████████████| 131 kB 55.1 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
     |████████████████████████████████| 1.2 MB 39.1 MB/s 
     |████████████████████████████████| 8.4 MB 22.0 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 


In [6]:
import torch
from pytorch_transformers import BertModel

bert_model = BertModel.from_pretrained("bert-base-uncased")
print(bert_model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [7]:
from pytorch_transformers import BertConfig

bert_config = BertConfig.from_pretrained("bert-base-uncased")
print(bert_config)

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pad_token_id": 0,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



## BertEmbeddings
* 入力を埋め込みベクトルへ変換する
* word_embeddings, position_embeddings, token_type_embeddingsの3つの埋め込みベクトルを足合わせる

## BertAttention
* BertSelfAttention, BertSelfOutputによりSelf-Attentionを実装
* Maskの実装

## BertIntermediate, BertOutput
* Positionwise fully connected feed-forward networkを実装

## BertPooler
* 最初のトークン(単語)を取得し全結合層、活性化関数で処理
* NextSentencePredictionなどで使用